In [61]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import requests


In [50]:
# Setup chrome driver for Chrome tool
def init_browser():
    #executable_path = {"executable_path": "C:/chromedriver/chromedriver"}
    executable_path = {'executable_path': ChromeDriverManager().install()}
    return Browser("chrome", **executable_path, headless=False)

In [51]:
# ------------------------------PART 1 START-------------------------------#

# Setup splinter
browser = init_browser()
#browser = Browser('chrome', **executable_path, headless=False)

# URL of page to be scraped
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

# Create BeautifulSoup object; parse with 'html.parser'
mars_news_soup = BeautifulSoup(response.text, 'html.parser')

# Grab title and content of the latest mars news
news_title = mars_news_soup.find("div", class_="content_title").find('a').text

news_content = mars_news_soup.find("div", class_="rollover_description_inner").text

# Close Browser.
browser.quit()

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88]


In [52]:
# Retrieve page with the requests module
#response = requests.get(url)
print(news_title + news_content)
# ------------------------------PART 1 END-------------------------------#


NASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities

Starting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.



In [57]:
# ------------------------------PART 2 START-------------------------------#
# JPL Mars Space Images - Featured Image
# Setup splinter
browser = init_browser()

jpl_mars_img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_mars_img_url)

# https://splinter.readthedocs.io/en/latest/api/driver-and-element-api.html
# use click_link_by_partial_text
browser.click_link_by_partial_text("FULL IMAGE")

# Have to click more info to open up bigger image, use find_link_by_partial_text again. Really useful function
browser.is_element_present_by_text("more info", wait_time=1)
bigger_img_elem = browser.find_link_by_partial_text("more info")
bigger_img_elem.click()

# Store the html object
img_html = browser.html

# Time to strip the image and store
mars_img_soup = BeautifulSoup(img_html, 'html.parser')

mars_img_url = mars_img_soup.find("figure", class_="lede").a["href"]

featured_image_url = f'https://www.jpl.nasa.gov{mars_img_url}'

# Close Browser.
browser.quit()

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


In [55]:
print(featured_image_url)
# ------------------------------PART 2 END-------------------------------#

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18905_hires.jpg


In [70]:
# ------------------------------PART 3 START-------------------------------#
# Mars Facts
# Much easier since just scraping a table, and we did a ton of that in the simpsons analysis for ETL project
mars_facts_url = "https://space-facts.com/mars/"

# use read_html
mars_facts_tables = pd.read_html(mars_facts_url)

mars_facts_df = mars_facts_tables[0]

# Set first column to Description as the prompt shows, and the second is Measurement/Units
mars_facts_df.columns = ['Description', 'Measurement/Units']

mars_facts_df

,Description,Measurement/Units
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [78]:
# To use on website we need it in html format
# Remove header just use html to produce header as it looks nicer
mars_facts_df.to_html('Resources/mars_facts.html', index=False, header=False)
mars_facts = mars_facts_df.to_html(index=False, header=False)
# ------------------------------PART 3 END-------------------------------#

In [74]:
# ------------------------------PART 4 START-------------------------------#
# Mars Hemispheres
browser = init_browser()

mars_hemis_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

mars_hemis_homepage = "https://astrogeology.usgs.gov";

browser.visit(mars_hemis_url)

# store into html object again
hemis_html = browser.html

hemis_soup = BeautifulSoup(hemis_html, 'html.parser')

# Grabs all 4 divs of the hemispheres
hemis_url = hemis_soup.find_all("div", class_="item")

# Store them as seperate html objects
hemis_url_all = []

for hemisphere in hemis_url:
    curr_hemis_url = hemisphere.find('a')['href']
    hemis_url_all.append(curr_hemis_url)

# Close Browser.
browser.quit()

print(hemis_url_all)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


 
['/search/map/Mars/Viking/cerberus_enhanced', '/search/map/Mars/Viking/schiaparelli_enhanced', '/search/map/Mars/Viking/syrtis_major_enhanced', '/search/map/Mars/Viking/valles_marineris_enhanced']


In [77]:
# Go through each link and store title and full link in a dict
mars_hemis_urls = []

for hemisphere in hemis_url_all:
    
    # create full link
    mars_url_full = mars_hemis_homepage + hemisphere
    
    # Abuse splinter again
    browser = init_browser()
    browser.visit(mars_url_full)
    
    hemis_html = browser.html
    hemis_soup = BeautifulSoup(hemis_html, 'html.parser')
    
    # Grab title
    hemis_title = hemis_soup.find("h2", class_="title").text
    
    # Grab the full.jpg for all the Hemispheres, aka the large size
    hemis_img_url = hemis_soup.find("li").a['href']
    
    mars_hemis_urls.append({'title': hemis_title, 'img_url': hemis_img_url})
    
    # Close Browser.
    browser.quit()

print(mars_hemis_urls)
# ------------------------------PART 4 END-------------------------------#

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\slobanwala\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


 
[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [80]:
# ------------------------------PART 5 START-------------------------------#
# Step 2 - MongoDB and Flask Application
mission_to_mars_data = {}

# Store all data
mission_to_mars_data['news_title'] = news_title
mission_to_mars_data['news_content'] = news_content
mission_to_mars_data['featured_image_url'] = featured_image_url
mission_to_mars_data['mars_facts'] = mars_facts
mission_to_mars_data['mars_hemisphere_urls'] = mars_hemis_urls

print(mission_to_mars_data)

{'news_title': '\nNASA to Broadcast Mars 2020 Perseverance Launch, Prelaunch Activities\n', 'news_content': '\nStarting July 27, news activities will cover everything from mission engineering and science to returning samples from Mars to, of course, the launch itself.\n', 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA14579_hires.jpg', 'mars_facts': '<table border="1" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n  

In [18]:
# ------------------------------PART 5 END-------------------------------#